# Phase Diagram for the Eutectic System

Here we develop a free energy functional and phase diagram for a system with a eutectic reaction $l \rightarrow \alpha + \beta$. The idea here is just to show that eutectic can still be done with this new model.

## Ideal Free Energy

---

The ideal free energy per unit area is of the following form

$$
\mathcal{F}_{id}[n] = \frac{1}{V_{cell}}\int_{cell} 
\frac{1}{2}n(x,y)^2 - \frac{\eta}{6} n(x,y)^3 + \frac{\chi}{12} n(x,y)^4
$$

To compute the phase diagram we use a variational method where we expand the reduced density, $n$, in the most general fourier series that is consistent with the symmetry of the bravais lattice.

$$
n(x,y) = n_0 + A_1 n_1(x,y) + A_2 n_2(x,y) + ...
$$

We then expand out the ansatz and compute each integral over the unit-cell numerically to find the amplitude equation for the ideal free energy

In [2]:
using PyPlot
using Interact
using Maxima
using Optim
using PhaseDiagrams

In [3]:
f_ideal = m"n^2/2 - η*n^3/6 + χ*n^4/12"

 
                                4      3      2
                               n  χ   n  η   n
                               ---- - ---- + --
                                12     6     2

In [4]:
n = m"n0 + 2*A*n1"            # + A[2]*n2"

f_ideal = subst(n, :n, f_ideal)

 
                          4                  3                  2
             (2 A n1 + n0)  χ   (2 A n1 + n0)  η   (2 A n1 + n0)
             ---------------- - ---------------- + --------------
                    12                 6                 2

In [5]:
f_ideal = expand(f_ideal)

 
    4   4        3      3                            3          4
 4 A  n1  χ   8 A  n0 n1  χ      2   2   2     2 A n0  n1 χ   n0  χ
 ---------- + ------------- + 2 A  n0  n1  χ + ------------ + -----
     3              3                               3          12
      3   3                                    3                              2
   4 A  n1  η      2      2         2        n0  η      2   2               n0
 - ---------- - 2 A  n0 n1  η - A n0  n1 η - ----- + 2 A  n1  + 2 A n0 n1 + ---
       3                                       6                             2

In [6]:
n1 = m"cos(4*%pi/sqrt(3)*(sqrt(3)*x/2 - y/2)) + 
       cos(4*%pi/sqrt(3)*(sqrt(3)*x/2 + y/2)) + 
       cos(4*%pi/sqrt(3)*y)"

n2 = m"cos(4*%pi*x) +
       cos(4*%pi*(x/2 + sqrt(3)*y/2)) + 
       cos(4*%pi*(-x/2 + sqrt(3)*y/2))"

f_ideal = subst(n1, :n1, f_ideal)
#f_ideal = subst(n2, :n2, f_ideal);

In [7]:
F_ideal = "integrate(integrate($f_ideal, x, 0, 1), y, 0, sqrt(3)/2)" |> MExpr;

In [8]:
F_ideal = mcall(F_ideal)

 
            3/2       4       7/2      2   2       5/2      3
 ((- ((- 2 3    %pi n0 ) - 8 3    %pi A  n0  - 32 3    %pi A  n0
       7/2      4           3/2       3      7/2      2          5/2      3
 - 20 3    %pi A ) χ) - (4 3    %pi n0  + 8 3    %pi A  n0 + 16 3    %pi A ) η
      5/2       2      7/2      2
 + 4 3    %pi n0  + 8 3    %pi A )/(144 %pi)

In [9]:
F_ideal = ratsimp(F_ideal)
F_ideal = "$F_ideal/(sqrt(3)/2)" |> MExpr |> mcall
F_ideal = ratsimp(F_ideal)
F_ideal = float(F_ideal)
F_ideal = expand(F_ideal)

 
                       4          2   2          3             4
 0.08333333333333333 n0  χ + 3.0 A  n0  χ + 4.0 A  n0 χ + 7.5 A  χ
                                3          2             3           2        2
         - 0.1666666666666667 n0  η - 3.0 A  n0 η - 2.0 A  η + 0.5 n0  + 3.0 A

In [10]:
F_id_exp = parse(F_ideal)                # Make julia expression of the ideal free energy
@eval F_id(η, χ, n0, A) = $F_id_exp     # Make julia function from the expression

F_id (generic function with 1 method)

## Free Energy of Mixing

The free energy of mixing is of the following form:

$$
    \mathcal{F}_{mix}[c] = \frac{1}{V_{cell}}\int_{cell} 
    (1 + n_0)\left(c \log\left(\frac{c}{c_0}\right) + 
    (1-c) \log\left(\frac{1-c}{1-c_0}\right)\right) + \frac{\epsilon(T)}{2}(c-c_0)^2
$$

In equilibrium we can assume that the concentration is constant so the integral just cancels the factor of the volume of the unit cell. The term $\epsilon(T)$ is the enthaply of mixing term which we model as, 

$$
\epsilon(T) = -4 + \epsilon_0 (T - T_0),
$$

where $T_0$ is the spinodal temperature of the liquid

In [11]:
 F_mix(c, T, n₀, ϵ₀, T₀, ω, c₀) = ω*((1+n₀)*(c*log(c/c₀) + (1-c)*log((1-c)/(1-c₀))) + (-4.0 + ϵ₀*(T-T₀))/2*(c-c₀)^2)

F_mix (generic function with 1 method)

## Correlation functions

Appropriate choice of $C_{nn}(k)$ for a eutectic model needs to be constructed. Using Nana's paper for the window functions and additions from Ken papers about ordered crystals for the temperature model

In [33]:
# Temperature 
T₀ = 1.0

kₐ = 2π
αₐ = 0.8

kᵦ = 4π/√3
αᵦ = 0.8

CA(k, T) = exp(- T / T₀) * exp(- (k - kₐ)^2 / (2 * αₐ^2))
CB(k, T) = exp(- T / T₀) * exp(- (k - kᵦ)^2 / (2 * αᵦ^2))
χ(c) = 2c^3 - 3c^2 + 1

Cₙₙ(k, c, T) = χ(c) * CA(k, T) + χ(1 - c) * CB(k, T)

function find_k(c, T)
    sol = optimize(k -> -Cₙₙ(k, c, T), 2π, 4π/√3)
    k₀ = float(sol.minimizer)
    return k₀
end

find_k (generic function with 1 method)

## Excess Free Energy

The excess free energy term in the binary phase field crystal model is the form

$$
\mathcal{F}_{ex}[n, c] = -\frac{1}{2 V_{cell}}\int_{cell} dr \int dr^\prime n(r) C_{eff}(r, r^\prime; c) n(r^\prime)
$$

Most of the modes in the amplitude expansion cancel out but we're left with the following at the end of the day:

$$
\mathcal{F}_{ex}[n, c] = -\frac{1}{2}(n_0^2 C_{eff}(k=0, c) + 6 A_1^2 C_{eff}(k=k_1, c) + 6 A_2^2 C_{eff}(k=k_2, c))
$$

In [13]:
function F_ex(c, T, A)
    k = c < 0.5 ? 2π : 4π/√3
    return (-3.0*A^2*Cₙₙ(k, c, T))       # + 6.0*A[2]^2*C_eff(k[2], c, T))
end

F_ex (generic function with 1 method)

## Total free energy

The total free energy is sum of all the above terms

In [34]:
F(c, T, η, χ, ϵ₀, T₀, ω, n₀, A, c₀) = F_mix(c, T, n₀, ϵ₀, T₀, ω, c₀) + F_ex(c, T, A)+ F_id(η, χ, n₀, A) 

F (generic function with 3 methods)

In [35]:
function F(c, T, η, χ, ϵ₀, T₀, ω, n₀, c₀)
    sol = optimize(A -> F(c, T, η, χ, ϵ₀, T₀, ω, n₀, A, c₀), 0.0, 10.0)
    #println("Amplitude at concentation $c and temperature $T is $(sol.minimum)")
    return sol.minimum
end

F (generic function with 3 methods)

In [80]:
pygui(false)

false

In [41]:
const η = 2.0
const χχ = 1.0
const ϵ₀ = 4.0 / 0.15
Tc = 0.15
const ω = 0.02
const n₀ = 0.05
const c₀ = 0.50

F(c, T) = F(c, T, η, χχ, ϵ₀, Tc, ω, n₀, c₀)

F (generic function with 3 methods)

In [42]:
function make_phase_diagram(F, crnge, Trnge)
    T_vals = Float64[] 
    c_vals = Float64[]
    
    c = collect(crnge)
    T = collect(Trnge)
    
    f = Array(Float64, length(c))
    
    for t in 1:length(T)
        for cc in 1:length(c)
            f[cc] = F(c[cc], T[t])
        end
        arr = commontangent(f)
        for domain in arr
            for point in domain
                push!(T_vals, T[t])
                push!(c_vals, c[point])
            end
        end
    end
    return T_vals, c_vals
end

make_phase_diagram (generic function with 2 methods)

In [48]:
crnge = 0.001:0.0005:0.999
Trnge = 0.00:0.010:1.0

T_vals, c_vals = make_phase_diagram(F, crnge, Trnge)

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] Array(::Type{Float64}, ::Int64) at ./deprecated.jl:57
 [3] make_phase_diagram(::#F, ::StepRangeLen{Float64,Base.TwicePrecision{Float64},Base.TwicePrecision{Float64}}, ::StepRangeLen{Float64,Base.TwicePrecision{Float64},Base.TwicePrecision{Float64}}) at ./In[42]:8
 [4] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /home/nsmith/.julia/v0.6/IJulia/src/execute_request.jl:160
 [5] eventloop(::ZMQ.Socket) at /home/nsmith/.julia/v0.6/IJulia/src/eventloop.jl:8
 [6] (::IJulia.##11#14)() at ./task.jl:335
while loading In[48], in expression starting on line 4


([0.0, 0.0, 0.01, 0.01, 0.02, 0.02, 0.03, 0.03, 0.04, 0.04  …  0.3, 0.3, 0.31, 0.31, 0.31, 0.31, 0.32, 0.32, 0.32, 0.32], [0.0105, 0.9895, 0.0115, 0.9885, 0.012, 0.988, 0.0135, 0.9865, 0.0145, 0.9855  …  0.8665, 0.9665, 0.017, 0.1035, 0.8965, 0.983, 0.003, 0.0615, 0.9385, 0.997])

In [49]:
Fₗ(c, T, η, χ, ϵ₀, T₀, ω, n₀) = F_mix(c, T, n₀, ϵ₀, T₀, ω, 0.5) + F_id(η, χ, n₀, 0.0) 
Fₗ(c, T) = Fₗ(c, T, η, χχ, ϵ₀, Tc, ω, n₀)

crange = 0.001:0.0005:0.999
trange = 0.00:0.010:0.60

T_b, c_b = make_phase_diagram(Fₗ, crange, trange)

function spinodal(c; n₀ = 0.05, ϵ₀=26.6666666666, T₀=Tc)
    return 1.0 / ϵ₀ * ((1 + n₀) / (c^2 - c) + 4.0) + T₀
end

T_s = spinodal.(0.0:0.02:1.0)
c_s = collect(0.0:0.02:1.0)

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] Array(::Type{Float64}, ::Int64) at ./deprecated.jl:57
 [3] make_phase_diagram(::#Fₗ, ::StepRangeLen{Float64,Base.TwicePrecision{Float64},Base.TwicePrecision{Float64}}, ::StepRangeLen{Float64,Base.TwicePrecision{Float64},Base.TwicePrecision{Float64}}) at ./In[42]:8
 [4] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /home/nsmith/.julia/v0.6/IJulia/src/execute_request.jl:160
 [5] eventloop(::ZMQ.Socket) at /home/nsmith/.julia/v0.6/IJulia/src/eventloop.jl:8
 [6] (::IJulia.##11#14)() at ./task.jl:335
while loading In[49], in expression starting on line 7


51-element Array{Float64,1}:
 0.0 
 0.02
 0.04
 0.06
 0.08
 0.1 
 0.12
 0.14
 0.16
 0.18
 0.2 
 0.22
 0.24
 ⋮   
 0.78
 0.8 
 0.82
 0.84
 0.86
 0.88
 0.9 
 0.92
 0.94
 0.96
 0.98
 1.0 

In [83]:
f = figure(figsize=(8, 6))
withfig(f) do
    scatter(c_vals, T_vals, s=5, label="liquid-solid coexistance")
    scatter(c_b, T_b, s=5, label="metastable liquid binodal")
    scatter(c_s, T_s, s=5, label="metastable liquid spinodal")
    legend(loc=0)
    grid(false)
    xlim(0, 1)
    ylim(0, 0.45)
    xlabel(L"Concentration $(c)$", fontsize=14)
    ylabel(L"Temperature $(T)$", fontsize=14)
end

savefig("Eutectic Phase Diagram.svg")

PyObject <matplotlib.text.Text object at 0x7f9aceb933d0>

In [103]:
scatter(c_vals, T_vals, c="k", s=10)
grid(false)
xlim(0, 1)
ylim(0, 0.38)
xlabel(L"Concentration $(c)$", fontsize=14)
ylabel(L"Temperature $(T)$", fontsize=14)
